In [1]:
import vaex
import glob
import pyarrow.parquet as pq
from vaex.dataframe import DataFrame
from datetime import datetime

In [5]:
%timeit -n 1 -r 1 vaex.open("./data/checkouts/Checkouts_By_Title_Data_Lens_*.csv",dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str", "ItemBarcode":"str","ItemType":"str"}, header=0).export_hdf5("./data/checkouts/tmp.hdf5")


7min 6s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%timeit -n 1 -r 1 vaex.from_csv("./data/checkouts/Checkouts.csv",dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str", "ItemBarcode":"str","ItemType":"str"}, header=0).export_hdf5("./data/checkouts/tmp.hdf5")


10min 21s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%timeit -n 1 -r 1 vaex.from_csv("./data/checkouts/Library_Collection_Inventory.csv", header=0).export_hdf5("./data/checkouts/tmp.hdf5")


19.9 s ± 1.64 s per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [2]:
def get_checkouts(file_type:str)->DataFrame:
    if file_type == "multi-csv":
        return vaex.open("./data/checkouts/Checkouts_By_Title_Data_Lens_*.csv", header=0, dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str", "ItemBarcode":"str","ItemType":"str"})
    if file_type == "single_csv":
        return vaex.from_csv("./data/checkouts/Checkouts.csv", header=0,dtype={"BibNum":"int64", "Title":"str","Author":"str", "Subjects":"str", "ItemBarcode":"str","ItemType":"str"})
    if file_type =="parquet":
        return vaex.open('./data/checkouts/checkouts_parquet/*.parquet')
    else:
        return vaex.open("./data/checkouts/checkouts.hdf5")

def get_inventory_no_dedup(file_type:str)->DataFrame:
    if "csv" in file_type:
        return vaex.from_csv("./data/checkouts/Library_Collection_Inventory.csv", header=0)
    if file_type =="parquet":
        return vaex.open('./data/checkouts/inventory_parquet/*.parquet')
    else:
        return vaex.open("./data/checkouts/inventory.hdf5")

def get_inventory(file_type:str)->DataFrame:
    if "csv" in file_type:
        df = vaex.open("./data/checkouts/Library_Collection_Inventory.csv")
        df = df.to_pandas_df()
        df = df.drop_duplicates('BibNum')
        return vaex.from_pandas(df)
    if file_type =="parquet":
        df = vaex.open('./data/checkouts/inventory_parquet/*.parquet')
        df = df.to_pandas_df()
        df = df.drop_duplicates('BibNum')
        return vaex.from_pandas(df)
    else:
        df = vaex.open("./data/checkouts/inventory.hdf5")
        df = df.to_pandas_df()
        df = df.drop_duplicates('BibNum')
        return vaex.from_pandas(df)

In [3]:
def writter_csv(df: DataFrame) -> None:
    df.export_csv('./output/output.csv', chunk_size=10000)
         
def writter_parquet(df: DataFrame) ->None:
    df.export_parquet("./output/parquet_output")
    
def writter_hdf5(df: DataFrame) ->None:
    df.export_hdf5("./output/hdf5_file.hdf5")

def to_dt(x):
    return datetime.strptime(x, "%m/%d/%Y %I:%M:%S %p")

def add_formatted_checkout(file_type:str)->DataFrame:
    df = get_checkouts(file_type)
    df["CheckoutDateTime_formatted"] = df.apply(to_dt, arguments=[df.CheckoutDateTime])
    return df

file_types = ["hdf5", "parquet", "single-csv"]# ,"multi-csv"]
wr = [writter_hdf5, writter_csv, writter_parquet ]

In [4]:
checkouts = get_checkouts("hdf5")[["BibNumber", "CheckoutDateTime"]]
inventory = get_inventory("hdf5")[["BibNum", "Author"]]


In [5]:
merged = checkouts.join(
        inventory, left_on="BibNumber", right_on="BibNum", how="inner"
    )

In [6]:
merged.head()

#,BibNumber,CheckoutDateTime,BibNum,Author
0,2.54365e+06,01/02/2017 08:13:00 AM,2.54365e+06,"Cohen, Leonard, 1934-2016"
1,3.1723e+06,01/02/2017 08:13:00 AM,3.1723e+06,"Cotterill, Colin"
2,3.19972e+06,01/02/2017 08:33:00 AM,3.19972e+06,None
3,3.21153e+06,01/02/2017 08:33:00 AM,3.21153e+06,Shonen Knife (Musical group)
4,2.74354e+06,01/02/2017 08:33:00 AM,2.74354e+06,"Manning, Shari Y."
5,3.10044e+06,01/02/2017 08:48:00 AM,3.10044e+06,"Marsh, Henry, 1950-"
6,2.6792e+06,01/02/2017 08:48:00 AM,2.6792e+06,"Mukherjee, Siddhartha"
7,3.16746e+06,01/02/2017 08:48:00 AM,3.16746e+06,"Hayden, Michael V. (Michael Vincent), 1945-"
8,3.17251e+06,01/02/2017 08:51:00 AM,3.17251e+06,None
9,3.21697e+06,01/02/2017 08:51:00 AM,3.21697e+06,None


In [8]:
inventory = vaex.from_pandas(inventory)

In [9]:
merged = checkouts.join(
        inventory,
        left_on="BibNumber",
        right_on="BibNum",
        how="inner",
        allow_duplication=True,
    )

In [ ]:
merged = merged.groupby(by = "Subject").agg({"BibNum":"nunique"})

In [7]:
    
    
    
    
    
    merged.head()

#,BibNumber,ItemBarcode,ItemType,Collection,CallNumber,CheckoutDateTime,CheckoutDateTime_formatted
0,2543647,0010063298235,accd,nacd,CD 782.42166 C6606So,01/02/2017 08:13:00 AM,2017-01-02 08:13:00
1,3172300,0010087522552,acbk,namys,MYSTERY COTTERI 2016,01/02/2017 08:13:00 AM,2017-01-02 08:13:00
2,2393405,0010054483200,acbk,camys,MYSTERY MAY2006,01/02/2017 08:24:00 AM,2017-01-02 08:24:00
3,3199718,0010088153514,acdvd,nadvdnf,DVD 781.66092 M3347G 2013,01/02/2017 08:33:00 AM,2017-01-02 08:33:00
4,3211526,0010089643810,accd,nacd,CD 782.42166 Sh75o,01/02/2017 08:33:00 AM,2017-01-02 08:33:00
...,...,...,...,...,...,...,...
"91,980,688",2286226,0010051021870,acbk,nafic,FIC BERRY2005,08/07/2008 03:48:00 PM,2008-08-07 15:48:00
"91,980,689",1978353,0010041313890,jcbk,ncenf,J628.92 BOELTS 1997,03/12/2008 03:14:00 PM,2008-03-12 15:14:00
"91,980,690",2441227,0010057847393,acbk,canf,747.78 L5161B 2007,12/12/2008 02:42:00 PM,2008-12-12 14:42:00
"91,980,691",2241685,0010049958126,acdvd,nadvd,DVD CASA_DE,05/13/2008 05:59:00 PM,2008-05-13 17:59:00


In [11]:
%timeit -n 1 -r 5 get_checkouts("parquet")
%timeit -n 1 -r 5 get_checkouts("multi-csv")
%timeit -n 1 -r 5 get_checkouts("single-csv")
%timeit -n 1 -r 5 get_checkouts("hdf5")

12.9 s ± 1.48 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
3min 12s ± 5.79 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
The slowest run took 8.13 times longer than the fastest. This could mean that an intermediate result is being cached.
29.2 ms ± 31.6 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
18.8 ms ± 9.41 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [12]:
%timeit -n 1 -r 5 get_checkouts("parquet").head()
%timeit -n 1 -r 5 get_checkouts("multi-csv").head()
%timeit -n 1 -r 5 get_checkouts("single-csv").head()
%timeit -n 1 -r 5 get_checkouts("hdf5").head()

10.4 s ± 333 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
3min 9s ± 922 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
21.8 ms ± 12 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
16.4 ms ± 2.95 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [4]:
for file_type in file_types:
    for writter in wr:
            print(f"read + write. file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 5 writter(get_inventory_no_dedup(file_type))
            print(f"Read, dedup, write. file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 5 writter(get_inventory(file_type))
            print(f"Get row count. file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 5 get_checkouts(file_type).count()
            #print(f"add timestamp, get first 10. file_type: {file_type}, writter: {writter.__name__}")
            #%timeit -n 1 -r 1 add_formatted_checkout(file_type).sort('CheckoutDateTime_formatted', ascending=True).head(10)
            print(f"Checkouts Read + Write. file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 1 writter(get_checkouts(file_type))
            print(f"Add timestamp and write. file_type: {file_type}, writter: {writter.__name__}")
            %timeit -n 1 -r 1 writter(add_formatted_checkout(file_type))



read + write. file_type: hdf5, writter: writter_hdf5
5.23 s ± 116 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
Read, dedup, write. file_type: hdf5, writter: writter_hdf5
7.33 s ± 250 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
Get row count. file_type: hdf5, writter: writter_hdf5
The slowest run took 4.48 times longer than the fastest. This could mean that an intermediate result is being cached.
57.6 ms ± 44.4 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
Checkouts Read + Write. file_type: hdf5, writter: writter_hdf5
1min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Add timestamp and write. file_type: hdf5, writter: writter_hdf5


ERROR:MainThread:root:error creating dataset for 'CheckoutDateTime_formatted', with type dtype('O') 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/vaex/hdf5/export.py", line 201, in export_hdf5
    array = h5column_output.require_dataset('data', shape=shape, dtype=dtype.newbyteorder(byteorder))
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/h5py/_hl/group.py", line 191, in require_dataset
    return self.create_dataset(name, *(shape, dtype), **kwds)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/h5py/_hl/group.py", line 136, in create_dataset
    dsid = dataset.make_new_dset(self, shape, dtype, data, **kwds)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/h5py/_hl/dataset.py", line 118, in make_new_dset
    tid = h5t.py_create(dtype, logical=1)
  File "h5py/h5t.pyx", line 1634, in h5py.h5t.py_creat

1min 50s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
read + write. file_type: hdf5, writter: writter_csv
31.3 s ± 2.54 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
Read, dedup, write. file_type: hdf5, writter: writter_csv
12.4 s ± 544 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
Get row count. file_type: hdf5, writter: writter_csv
53.5 ms ± 34.7 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
Checkouts Read + Write. file_type: hdf5, writter: writter_csv
8min 3s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Add timestamp and write. file_type: hdf5, writter: writter_csv


KeyboardInterrupt: 

In [ ]:
for file_type in file_types:
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            df = add_formatted_checkout(file_type)
            df = df.CheckoutTime_formated.min()
            print(df)
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory("parquet")[["BibNum", "Author"]]
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="outer", indicator=True)
        merged[merged._merge=="right_only"].head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    for writter in wr:
        stop_list = []
        for i in range(5):
            start = timeit.default_timer()
            inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
            inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
            inventory = inventory.explode("Subject")
            writter(inventory)
            stop = timeit.default_timer()
            stop_list = stop_list.append(stop-start)
        print(f"file_type: {file_type}, writter: {writter.__name__}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        inventory = inventory.groupby(inventory.Subject)["BibNum"].nunique().astype("int64"))
        inventory.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(5):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.groupby(merged.Subject)["BibNum"].nunique().astype("int64")
        merged.head()
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, writter: {writter.__name__}, runtime: {sum(stop_list)/len(stop_list)}")

In [ ]:
for file_type in file_types:
    stop_list = []
    for i in range(1):
        start = timeit.default_timer()
        checkouts = get_checkouts("parquet")[["BibNumber", "CheckoutDateTime"]]
        inventory = get_inventory(file_type)[["BibNum", "Subjects"]]
        inventory['Subject'] = inventory['Subjects'].str.split(',').fillna("")
        inventory = inventory.explode("Subject")
        merged = dd.merge(checkouts, inventory, left_on="BibNumber", right_on="BibNum", how="inner")
        merged = merged.groupby(merged.index)["BibNum"].nunique().astype("int64").sort("BibNum", ascending=False)
        merged.head(10)
        stop = timeit.default_timer()
        stop_list = stop_list.append(stop-start)
    print(f"file_type: {file_type}, writter: {writter.__name__}, runtime: {sum(stop_list)/len(stop_list)}")